In [6]:
import imageio.v2 as imageio
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import cv2
import numpy as np
import glob
import os


CHECKERBOARD = (7, 7)
SQUARE_SIZE = 1.45

IMG_DIR = "/content/drive/MyDrive/Amazon project/dng_images"  # ★ JPG フォルダに変更

# チェッカーボード3D座標
objp = np.zeros((CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[:, :2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
objp *= SQUARE_SIZE

objpoints = []  # 3Dポイント
imgpoints = []  # 2Dポイント


jpg_files = glob.glob(os.path.join(IMG_DIR, "*.jpg"))
print(f"Found {len(jpg_files)} JPG files.")

if len(jpg_files) == 0:
    print("JPGが1枚も見つかりません。パスを確認してください。")

for fname in jpg_files:

    img = cv2.imread(fname)
    if img is None:
        print(f"✖ Failed to read: {fname}")
        continue

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # チェッカーボード検出
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, None)

    if ret:
        # コーナーの精緻化
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
        corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)

        objpoints.append(objp)
        imgpoints.append(corners2)

        print(f"✔ Detected corners: {os.path.basename(fname)}")
    else:
        print(f"✖ Failed: {os.path.basename(fname)}")



ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(
    objpoints, imgpoints, gray.shape[::-1], None, None
)

print("\n===== Calibration Results =====")
print("Intrinsic Matrix K:\n", K)
print("Distortion Coefficients:\n", dist)
print("Reprojection Error:", ret)


total_error = 0
errors = []

for i in range(len(objpoints)):
    imgpoints2, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
    error = cv2.norm(imgpoints[i], imgpoints2, cv2.NORM_L2) / len(imgpoints2)
    errors.append(error)
    total_error += error

mean_error = total_error / len(objpoints)

print("\n=====  Evaluation =====")
for i, e in enumerate(errors):
    print(f"Image {i+1}: {e:.4f} px")

print(f"\nMean Reprojection Error: {mean_error:.4f} px")

np.save("intrinsic.npy", K)
np.save("distortion.npy", dist)

print("\nSaved: intrinsic.npy / distortion.npy")


Found 43 JPG files.
✔ Detected corners: 7.jpg
✔ Detected corners: 6.jpg
✔ Detected corners: 5.jpg
✔ Detected corners: 22.jpg
✔ Detected corners: 10.jpg
✔ Detected corners: 23.jpg
✔ Detected corners: 18.jpg
✔ Detected corners: 17.jpg
✔ Detected corners: 15.jpg
✔ Detected corners: 19.jpg
✔ Detected corners: 21.jpg
✔ Detected corners: 16.jpg
✔ Detected corners: 20.jpg
✔ Detected corners: 9.jpg
✔ Detected corners: 8.jpg
✔ Detected corners: 12.jpg
✔ Detected corners: 14.jpg
✔ Detected corners: 4.jpg
✔ Detected corners: 11.jpg
✔ Detected corners: 1.jpg
✔ Detected corners: 13.jpg
✔ Detected corners: 3.jpg
✔ Detected corners: 2.jpg
✔ Detected corners: S__121413669_0.jpg
✔ Detected corners: S__121413671_0.jpg
✔ Detected corners: S__121413672_0.jpg
✔ Detected corners: S__121413673_0.jpg
✔ Detected corners: S__121413674_0.jpg
✔ Detected corners: S__121413678_0.jpg
✔ Detected corners: S__121413676_0.jpg
✔ Detected corners: S__121413670_0.jpg
✔ Detected corners: S__121413675_0.jpg
✔ Detected corner